In [6]:
# bob.refresh {"cron": "5,30 2,3,4,5,6,19,20,21 * * *", "timeout": 40000}

In [4]:
import bob
hc = bob.HealthCheck()
hc_key = "51c8f3e8-3d84-4888-878e-a53f0a152e1a"
start = hc.start(hc_key)
print(start)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/51c8f3e8-3d84-4888-878e-a53f0a152e1a, 2020-04-06


In [2]:
import bob
import json
from datetime import datetime, timedelta, date
import time
import pandas as pd
import numpy as np

#--- STEP0 --- : Init
#-> Repository
input_folder = "../data_input/"
output_folder = "../data_output/"
param_folder = "../../_parameters/"

#-> Date
now = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

#-> Get credentials info+
credentials = param_folder + 'mongodb.json'
with open(credentials) as f:
    dbCon = json.load(f)

mongo_host = dbCon["host"]
mongo_port = dbCon["port"]
mongo_username = dbCon["username"]
mongo_password = dbCon["password"]

#-> Bob connect
MongoDB = bob.mongo.connect(mongo_host,mongo_port,mongo_username,mongo_password)
db = 'app-wsr-v1'

Successfully connected to MongoDB


In [4]:
#--- Load data source
start_time = time.time()
db_conso = pd.read_csv(output_folder + 'WORLD_DB_CONSO.csv', sep=';',low_memory=False)
db_all = pd.read_csv(output_folder + 'WORLD_DB_ALL.csv', sep=';',low_memory=False)
db_trend = pd.read_csv(output_folder + 'WORLD_DB_TREND.csv', sep=';',low_memory=False)
db_all_fr = pd.read_csv(output_folder + 'FRANCE_DB_ALL.csv', sep=';',low_memory=False)
# db_conso = bob.mongo.read_df('WORLD_DB_CONSO','covid-19').drop('_id', axis=1)
# db_all = bob.mongo.read_df('WORLD_DB_ALL','covid-19').drop('_id', axis=1)
# db_trend = bob.mongo.read_df('WORLD_DB_TREND','covid-19').drop('_id', axis=1)
# db_all_fr = bob.mongo.read_df('FRANCE_DB_ALL','covid-19').drop('_id', axis=1)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 04/04/2020 11:11:56. Time: --- 2.6272594928741455 secnds ---


In [9]:
#-- Date selector : ENTITIES
def get_entities(df):
    cols_to_rename = {"ENTITY": "ENTITIES", "ENTITY_GROUPS": "GROUPS"}
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df = df[df['ENTITY_GROUPS'].isin(filters)].rename(index=str, columns=cols_to_rename)[['ENTITIES','GROUPS']].drop_duplicates().sort_values(by="ENTITIES", ascending=True).reset_index(drop=True)
    df['ORDER'] = df.index + 8
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER'] = 8
    
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER_GROUPS'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER_GROUPS'] = 8

    df.loc[df['GROUPS'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['GROUPS'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['GROUPS'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['GROUPS'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['GROUPS'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['GROUPS'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['GROUPS'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['GROUPS'] == 'To be affected', 'ORDER_GROUPS'] = 8
    
    df.loc[df['GROUPS'] == 'WORLDWIDE','ENTITY_DISPLAY'] = df['ENTITIES']
    df.loc[df['GROUPS'] != 'WORLDWIDE','ENTITY_DISPLAY'] = '--- ' + df['ENTITIES']
    return df.sort_values(by=['ORDER_GROUPS','ORDER'], ascending=True)

entities = get_entities(db_conso)
entities.to_csv(output_folder + 'ENTITIES.csv',sep=";")
bob.mongo.save_df(entities,'ENTITIES',db,True)
# entities.head(5)

Dataframe ENTITIES successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.02267289161682129 secnds ---


In [10]:
#-- Date selector : SCENARIOS
def get_scenarios(df):
    df = df[['SCENARIO','DATE_ORDER']].drop_duplicates()
    df['SCENARIO_DIPLAY'] = pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y').dt.strftime('%d %b %Y')
    return df.sort_values(by='DATE_ORDER', ascending=False)

scenarios = get_scenarios(db_all)
scenarios.to_csv(output_folder + 'SCENARIOS.csv',sep=";")
bob.mongo.save_df(scenarios,'SCENARIOS',db,True)
scenarios.head(5)

Dataframe SCENARIOS successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.02179408073425293 secnds ---


,SCENARIO,DATE_ORDER,SCENARIO_DIPLAY
72,03/04/2020,20200403,03 Apr 2020
71,02/04/2020,20200402,02 Apr 2020
70,01/04/2020,20200401,01 Apr 2020
69,31/03/2020,20200331,31 Mar 2020
68,30/03/2020,20200330,30 Mar 2020


In [11]:
#Dashboard value dynamic template
def dashboard_value(df,kpi,label):
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain = df[(df['ENTITY_GROUPS'].isin(filters)) & (df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain[label],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': now,
    })
    return domain

In [12]:
#Dashboard leaderboard template
def dashboard_leaderboard(df,kpi):
    domain = df[(df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['ENTITY'],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': now,
    })
    return domain

In [13]:
#Duplicate to rank countries WORLDWIDE
def get_ww(df): 
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df_ww = df.copy()[(df['ENTITY'].isin(filters))] 
    df_ww['ENTITY'] = "WORLDWIDE"           
    df = df.append(df_ww)
    return df

In [14]:
#--- DASHBOARD ---
#-- VALUE DYNAMIC KPIS
start_time = time.time()
domain001 = dashboard_value(db_all,'Confirmed','KPI')
domain001.to_csv(output_folder + '001.csv',sep=";")
bob.mongo.save_df(domain001,'001', db,True)

domain002 = dashboard_value(db_all,'Deaths','KPI')
domain002.to_csv(output_folder + '002.csv',sep=";")
bob.mongo.save_df(domain002,'002', db,True)

domain003 = dashboard_value(db_all,'Recovered','KPI')
domain003.to_csv(output_folder + '003.csv',sep=";")
bob.mongo.save_df(domain003,'003', db,True)

domain004 = dashboard_value(db_all,'Active cases','KPI')
domain004.to_csv(output_folder + '004.csv',sep=";")
bob.mongo.save_df(domain004,'004', db,True)

domain005 = dashboard_value(db_all,"Fatality Rate",'KPI')
domain005.to_csv(output_folder + '005.csv',sep=";")
bob.mongo.save_df(domain005,'005', db,True)

domain006 = dashboard_value(db_all,"Recovery Rate",'KPI')
domain006.to_csv(output_folder + '006.csv',sep=";")
bob.mongo.save_df(domain006,'006', db,True)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))

Dataframe 001 successfully save in database app-wsr-v1 in MongoDB. Time: --- 10.344755172729492 secnds ---
Dataframe 002 successfully save in database app-wsr-v1 in MongoDB. Time: --- 10.138439655303955 secnds ---
Dataframe 003 successfully save in database app-wsr-v1 in MongoDB. Time: --- 9.976490497589111 secnds ---
Dataframe 004 successfully save in database app-wsr-v1 in MongoDB. Time: --- 10.451014757156372 secnds ---
Dataframe 005 successfully save in database app-wsr-v1 in MongoDB. Time: --- 10.278908967971802 secnds ---
Dataframe 006 successfully save in database app-wsr-v1 in MongoDB. Time: --- 9.818326950073242 secnds ---
Script execution completed at 04/04/2020 07:14:26. Time: --- 61.86918616294861 secnds ---


In [15]:
#-- LEADERBOARD BY REGION
start_time = time.time()
db_r = db_all[(db_all['ENTITY_GROUPS'] == 'WORLDWIDE') & (db_all['ENTITY'] != 'WORLDWIDE')]
domain007 = dashboard_leaderboard(db_r,'Confirmed')
domain007.to_csv(output_folder + '007.csv',sep=";")
bob.mongo.save_df(domain007,'007', db,True)

domain008 = dashboard_leaderboard(db_r,'Deaths')
domain008.to_csv(output_folder + '008.csv',sep=";")
bob.mongo.save_df(domain008,'008', db,True)

domain009 = dashboard_leaderboard(db_r,'Recovered')
domain009.to_csv(output_folder + '009.csv',sep=";")
bob.mongo.save_df(domain009,'009', db,True)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))

Dataframe 007 successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.0447232723236084 secnds ---
Dataframe 008 successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.04421401023864746 secnds ---
Dataframe 009 successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.04503488540649414 secnds ---
Script execution completed at 04/04/2020 07:14:26. Time: --- 0.18891453742980957 secnds ---


In [16]:
#-- LEADERBOARD BY COUNTRIES
start_time = time.time()
db_c = db_all[(db_all['ENTITY_GROUPS'] != 'WORLDWIDE')]
domain010 = get_ww(dashboard_leaderboard(db_c,'Confirmed'))
domain010.to_csv(output_folder + '010.csv',sep=";")
bob.mongo.save_df(domain010,'010',db,True)

domain011 = get_ww(dashboard_leaderboard(db_c,'Deaths'))
domain011.to_csv(output_folder + '011.csv',sep=";")
bob.mongo.save_df(domain011,'011', db,True)

domain012 = get_ww(dashboard_leaderboard(db_c,'Recovered'))
domain012.to_csv(output_folder + '012.csv',sep=";")
bob.mongo.save_df(domain012,'012', db,True)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))

Dataframe 010 successfully save in database app-wsr-v1 in MongoDB. Time: --- 19.519050121307373 secnds ---
Dataframe 011 successfully save in database app-wsr-v1 in MongoDB. Time: --- 19.518616199493408 secnds ---
Dataframe 012 successfully save in database app-wsr-v1 in MongoDB. Time: --- 19.156473398208618 secnds ---
Script execution completed at 04/04/2020 07:14:26. Time: --- 59.41916036605835 secnds ---


In [17]:
#-- Covid-19 - TRENDS
start_time = time.time()
def domain_101(df):
    domain = df[df['KPI'].isin(['Confirmed','Deaths','Recovered','Active cases','Deaths ratio','Recovered ratio'])]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'GROUPS': domain['KPI'],
        'BOTTOM_FILTER': domain['DATE_SCENARIO'],
        'UPPER_FILTER_M': domain['METRIC'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
        'LAST_UPDATE': now,
    })
    
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases'])),'UPPER_FILTER_M'] = 'DAILY VARIATION (CASES)'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = 'DAILY VARIATION (PTS)'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases'])),'UPPER_FILTER_M'] = 'DAILY VARIATION (%)'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = ''
    return domain

domain101 = domain_101(db_trend)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain101.to_csv(output_folder + '101.csv',sep=";")
bob.mongo.save_df(domain101,'101',db,True)
# domain101.head(5)

Script execution completed at 04/04/2020 07:14:26. Time: --- 0.5818274021148682 secnds ---
Dataframe 101 successfully save in database app-wsr-v1 in MongoDB. Time: --- 55.88365626335144 secnds ---


In [18]:
#-- Covid-19 - TRENDS BY COUNTRIES
start_time = time.time()
def domain_102(df):
    domain = df.copy()
    max_date = domain['DATE_ORDER'].max()
    filters =  ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    countries = domain[(domain['ENTITY_GROUPS'].isin(filters)) & (domain['DATE_ORDER'] == max_date) & (domain['GROUPS'] == 'Confirmed') & (domain['BOTTOM_FILTER'] == 'Since beginning')].sort_values(by= 'VALUE', ascending=False)['ENTITY'].drop_duplicates().values.tolist()[:10]
    domain = domain[(domain['ENTITY_GROUPS'] != "WORLDWIDE") & (domain['ENTITY'].isin(countries))]
    return domain

domain102 = domain_102(domain101)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain102.to_csv(output_folder + '102.csv',sep=";")
bob.mongo.save_df(domain102,'102',db,True)
# domain102.head(5)

Script execution completed at 04/04/2020 07:14:26. Time: --- 0.22937583923339844 secnds ---
Dataframe 102 successfully save in database app-wsr-v1 in MongoDB. Time: --- 18.474308490753174 secnds ---


In [19]:
#-- Covid-19 - RANKING BY COUNTRIES
start_time = time.time()
#-> Referentials
ref_continent = pd.read_excel(input_folder + 'REF_WSR.xlsx', sheet_name = 'REF_CONTINENT')
def domain_103(df,ref):
    domain = df.copy()
    cols_to_rename = {"LABEL": "KPI","ENTITY": "LABEL", "ENTITY_GROUPS":"ENTITY", "VARV":"VAR"}
    domain = get_ww(domain.drop(['DATE','UNIT_VAR','UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP'],axis=1).rename(index=str, columns=cols_to_rename).reset_index(drop=True))
    domain = pd.merge(domain, ref, left_on=['LABEL'], right_on=['COUNTRY_REGION'], how='left').drop(['CONTINENT','COUNTRY_REGION'],axis=1)
    return domain.reset_index(drop=True)

domain103 = domain_103(db_all,ref_continent)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain103.to_csv(output_folder + '103.csv',sep=";")
bob.mongo.save_df(domain103,'103',db,True)
# domain103.head(5)

Script execution completed at 04/04/2020 07:14:26. Time: --- 0.4764735698699951 secnds ---
Dataframe 103 successfully save in database app-wsr-v1 in MongoDB. Time: --- 41.83936357498169 secnds ---


In [5]:
#-- Covid-19 - FRANCE FOCUS
start_time = time.time()
def domain_105(df):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'LABEL': domain['LABEL'],
        'GROUPS': domain['LABEL_GROUPS'],
        'GROUPS_ORDER': domain['STATUS_ORDER'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'BOTTOM_FILTER': 'Depuis le début',
        'UPPER_FILTER_M': domain['STATUS_NAME'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': ' ',
        'PRECISION': ',.0f',
        'LAST_UPDATE': now,
    })
    
    domain = domain.sort_values(by='DATE_ORDER',ascending=True)
    def get_lastdays(df,all_dates,title):
        df_b = pd.DataFrame()
        for d in all_dates:
            tmp_df = df[df['DATE_ORDER'] == d]
            df_b = df_b.append(tmp_df)
            df_b['BOTTOM_FILTER'] = title
        return df_b
    
    last_14 = domain['DATE_ORDER'].sort_values(ascending=False).drop_duplicates().values.tolist()[:14]
    df_last14 = get_lastdays(domain, last_14,"14 derniers jours")
    last_30 = domain['DATE_ORDER'].sort_values(ascending=False).drop_duplicates().values.tolist()[:30]
    df_last30 = get_lastdays(domain, last_30,"30 derniers jours")
    domain = pd.concat([domain,df_last14,df_last30],axis=0)
    
    return domain.reset_index(drop=True)

domain105 = domain_105(db_all_fr).sort_values(by=['DATE_ORDER','LABEL','GROUPS','GROUPS_ORDER'],ascending=True)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain105.to_csv(output_folder + '105.csv', sep=';')
bob.mongo.save_df(domain105,'105',db,True)
# domain105.head(5)

Script execution completed at 04/04/2020 11:11:56. Time: --- 0.19437956809997559 secnds ---
Dataframe 105 successfully save in database app-wsr-v1 in MongoDB. Time: --- 15.282022714614868 secnds ---


In [21]:
#-- Covid-19 - FRANCE FOCUS
start_time = time.time()
def domain_106(df):
    df = df[~df['KPI'].isin(['Fatality Rate',"Recovery Rate"])]
    
    #--Value
    domain_val = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VALUE','VALUE_D-1']][df['ENTITY_GROUPS'] == 'WORLDWIDE']
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI']
    domain_val = domain_val.melt(id_vars=cols_to_keep, value_name='VALUE',var_name='METRIC')
    domain_val['ENTITY_GROUPS'] = domain_val['ENTITY']
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'LABEL_ORDER'] = 99999999
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'LABEL_ORDER'] = 0
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'METRIC'] = domain_val['SCENARIO']
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'METRIC'] = (pd.to_datetime(domain_val['SCENARIO'], format='%d/%m/%Y') + timedelta(days=-1)).dt.strftime('%d/%m/%Y')
    
    cols_to_rename = {'METRIC':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_val = domain_val.drop('ENTITY',axis=1).rename(index=str, columns=cols_to_rename)
    domain_val['PRECISION'] = ',.0f'

    #--Variation
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain_var = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VARV','VARP']][df['ENTITY_GROUPS'].isin(filters)]
    cols_to_rename = {'VARV':'VALUE', 'VARP': 'VAR'}
    domain_var = domain_var.rename(index=str, columns=cols_to_rename).reset_index(drop=True)
    domain_var = domain_var[domain_var['VALUE'] !=0 ]
    
    filters = domain_var[['ENTITY_GROUPS','SCENARIO','KPI']].drop_duplicates().values.tolist()
    domain_w = pd.DataFrame()
    for f in filters:
        tmp_df = domain_var[(domain_var['ENTITY_GROUPS'] == f[0]) & (domain_var['SCENARIO'] == f[1]) & (domain_var['KPI'] == f[2])].sort_values(by=["VALUE","ENTITY"],ascending=False).reset_index(drop=True)
        tmp_df['LABEL_ORDER'] = tmp_df.index.astype(int) + 1
        if len(tmp_df) > 9:
            tmp_df.loc[tmp_df.index > 9, 'ENTITY'] = 'Other countries'
            cols_to_group = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','LABEL_ORDER']
            tmp_df = tmp_df.groupby(cols_to_group, as_index=False).agg({'VALUE':'sum'})
        tmp_df['PRECISION'] = '+,.0f'
        domain_w = domain_w.append(tmp_df)
        
    cols_to_rename = {'ENTITY':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_w = domain_w.rename(index=str, columns=cols_to_rename)
    domain_w['GROUPS'] = domain_w['LABEL']
    
    cols_to_rename = {'LABEL':'GROUPS', 'ENTITY':'LABEL'}
    domain_w2 = domain_w.copy().drop('GROUPS',axis=1).rename(index=str, columns=cols_to_rename)
    domain_w2['ENTITY'] = 'WORLDWIDE'
    
    domain = pd.concat([domain_val,domain_w,domain_w2],axis=0).sort_values(['ENTITY','UPPER_FILTER_R','LABEL_ORDER'])
    return domain.reset_index(drop=True)

domain106 = domain_106(db_all)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))
# domain106 = domain106[(domain106['UPPER_FILTER_R'] == 'Confirmed') & (domain106['SCENARIO'] == '02/04/2020')]
domain106.to_csv(output_folder + '106.csv', sep=";")
bob.mongo.save_df(domain106,'106',db,True)
# domain106.head(15)

Script execution completed at 04/04/2020 07:14:26. Time: --- 12.703436613082886 secnds ---
Dataframe 106 successfully save in database app-wsr-v1 in MongoDB. Time: --- 18.11886739730835 secnds ---


In [5]:
done = hc.done(hc_key)
print(done)

End ==> send to https://health.cashstory.com/ping/51c8f3e8-3d84-4888-878e-a53f0a152e1a, 2020-04-06
